In [1]:
# Clean data for modeling

pitch = pd.read_csv('../data/mlb-pitches.csv', index_col = [0])
pitch = pitch[['player_name', 'p_throws', 'pitch_type','release_speed', 'release_spin_rate', 'spin_axis', 
               'pfx_-x', 'pfx_z', 'bauer_units', 'effective_speed', 'release_pos_x', 'release_pos_z', 
               'release_extension', 'release_pos_y', 'plate_-x', 'plate_x', 'plate_z', 'type', 'balls','strikes', 
               'pitch_count', 'stand', 'description', 'events', 'hit_distance_sc', 'launch_speed','launch_angle', 
               'launch_speed_angle', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 
               'woba_value', 'woba_denom', 'babip_value', 'iso_value','at_bat_number', 'pitch_number', 
               'inning', 'inning_topbot', 'home_score', 'away_score', 'post_home_score', 'post_away_score', 
               'on_1b', 'on_2b', 'on_3b', 'outs_when_up', 'delta_run_exp']].copy()

#Rename some columns
col_dict = {
    'release_speed': 'velo',
    'release_spin_rate': 'spin_rate',
    'launch_speed': 'exit_velo',
    'estimated_ba_using_speedangle': 'xba',
    'estimated_woba_using_speedangle': 'xwobacon'
}
pitch.rename(columns = col_dict, inplace = True)
pitch.to_csv('../data/model-pitches.csv')

In [7]:
# Add RV to model-pitches.csv

data = pd.read_csv('../data/model-pitches.csv', index_col = [0])
data.dropna(subset = ['pitch_type', 'velo', 'spin_rate', 'pfx_-x', 
                      'release_extension', 'delta_run_exp'], inplace = True)

data['inning_topbot'] = data.inning_topbot.map({'Top': 0, 'Bot': 1})
data['on_1b'] = [1 if x > 1 else 0 for x in data['on_1b']]
data['on_2b'] = [1 if x > 1 else 0 for x in data['on_2b']]
data['on_3b'] = [1 if x > 1 else 0 for x in data['on_3b']]

data['home_runs'] = data['post_home_score'] - data['home_score']
data['away_runs'] = data['post_away_score'] - data['away_score']
data['runs'] = data['home_runs'] + data['away_runs']

re = pd.read_csv('../data/run_expectancy_table.csv', index_col = [0])
data = pd.merge(data, re, how = 'left', on = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up'])

data['re_end_state'] = data['delta_run_exp'] + data['re']
data['re24'] = data['re_end_state'] - data['re'] + data['runs']

count_woba = pd.read_csv('../data/count_woba_value.csv', index_col = [0])
data = pd.merge(data, count_woba, how = 'left', on = ['pitch_count'])
data['count_woba_diff'] = round(-data['count_woba_value'].diff(1), 3)
data['count_woba_after'] = data['count_woba_value'] + data['count_woba_diff']
data['count_woba_diff'].fillna(0, inplace = True)
data['count_woba_after'].fillna(0, inplace = True)
data['xRV'] = round((data['count_woba_diff']) / 1.15, 3)

pd.set_option('max_columns', None)
print(data.shape)
data.head(10)
#data.to_csv('../data/model-pitches-rv.csv')

(705403, 57)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,stand,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,xba,xwobacon,woba_value,woba_denom,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,delta_run_exp,home_runs,away_runs,runs,re,re_end_state,re24,count_woba_value,count_woba_diff,count_woba_after,xRV
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,R,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.174,0.158,0.0,1.0,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,-0.073,0,0,0,0.115,0.042,-0.073,0.233,0.000,0.000,0.000
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,1-1,R,foul,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,-0.027,0,0,0,0.115,0.088,-0.027,0.313,-0.080,0.233,-0.070
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,1-0,R,foul,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,-0.020,0,0,0,0.115,0.095,-0.020,0.374,-0.061,0.313,-0.053
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0-0,R,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,0.016,0,0,0,0.115,0.131,0.016,0.331,0.043,0.374,0.037
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,L,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.100,0.090,0.0,1.0,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,-0.189,0,0,0,0.298,0.109,-0.189,0.374,-0.043,0.331,-0.037
5,"Smith, Will",L,FF,92.5,2319.0,145.0,-6.48,17.40,25.070270,92.0,1.35,6.74,6.3,54.19,1.04,-1.04,3.58,B,0,0,0-0,L,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,1,9,0,5,0,5,0,0,0,0,1,0.026,0,0,0,0.298,0.324,0.026,0.331,0.043,0.374,0.037
6,"Smith, Will",L,SL,80.3,2328.0,304.0,3.96,2.16,28.991283,80.8,1.81,6.55,6.2,54.30,0.69,-0.69,0.83,S,0,2,0-2,L,swinging_strike_blocked,strikeout,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,59,3,9,0,5,0,5,0,0,0,0,0,-0.152,0,0,0,0.544,0.392,-0.152,0.207,0.124,0.331,0.108
7,"Smith, Will",L,SL,79.5,2368.0,306.0,6.00,4.56,29.786164,79.4,1.72,6.56,6.1,54.44,-0.12,0.12,1.99,S,0,1,0-1,L,called_strike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,2,9,0,5,0,5,0,0,0,0,0,-0.052,0,0,0,0.544,0.492,-0.052,0.281,-0.074,0.207,-0.064
8,"Smith, Will",L,FF,91.8,2299.0,146.0,-8.88,17.64,25.043573,91.8,1.39,6.75,6.5,54.04,0.72,-0.72,2.77,S,0,0,0-0,L,called_strike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,1,9,0,5,0,5,0,0,0,0,0,-0.038,0,0,0,0.544,0.506,-0.038,0.331,-0.050,0.281,-0.043
9,"Gsellman, Robert",R,SI,94.3,1982.0,221.0,16.20,10.56,21.018028,94.4,-1.20,6.05,6.5,53.98,0.26,-0.26,1.78,X,0,2,0-2,R,hit_into_play,field_out,140.0,75.3,65.0,3.0,0.000,0.000,0.0,1.0,0.0,0.0,58,3,8,1,5,0,5,0,0,0,0,2,-0.061,0,0,0,0.115,0.054,-0.061,0.207,0.124,0.331,0.108


In [6]:
data.count_woba_diff.mean()

-1.998857390742549e-07

In [8]:
data.xRV.mean()

3.690372737281557e-05